In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install MTCNN

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [3]:
import cv2
from mtcnn.mtcnn import MTCNN
import urllib.request
from urllib.error import HTTPError
import pandas as pd
import numpy as np
import progressbar
from google.colab import files
batch = "d8"

Using TensorFlow backend.


In [0]:
data = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/"+batch+".pkl")
#data = data[['listing_id', 'id', 'pic']]

In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [0]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	return image

In [0]:
def detect_face_mtcnn(frame):
    imgtest1 = frame.copy()
    # convert to RGB, because OpenCV uses BGR -> mtcnn uses RGB
    imgplot = cv2.cvtColor(imgtest1, cv2.COLOR_BGR2RGB)
    # detect faces in the image
    resultList = detector.detect_faces(imgplot)
    
    faceBoxes=[]
    points=[]
    if(len(resultList)==1):
        # draw the box
        x, y, width, height = resultList[0]['box']
        x1=x
        y1=y
        x2=x+width
        y2=y+height
        faceBoxes.append([x1,y1,x2,y2])
        points.append(resultList[0]['keypoints'])
    return len(resultList), faceBoxes, points

In [0]:
def analyze_pic(url):
    frame = url_to_image(url) #import pic from the local folder
    #get box coordination locating faces
    face_num, faceBoxes, points = detect_face_mtcnn(frame)

    gender_ret = []
    age_ret = [] 
    genderPreds=np.nan
    agePreds=np.nan
    #predict only when theres only one face in the pic
    if (face_num==1):
        #crop only the face part
        faceBox = faceBoxes[0]
        face = frame[max(0,faceBox[1]-padding):min(faceBox[3]+padding,frame.shape[0]-1),
                     max(0,faceBox[0]-padding):min(faceBox[2]+padding, frame.shape[1]-1)]
        
        #preprocessing
        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
        
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        gender_ret.append(gender)

        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=ageList[agePreds[0].argmax()]
        age_ret.append(age)
    return frame, face_num, age_ret, gender_ret, agePreds, genderPreds, faceBoxes, points


In [0]:
#initialize ML var
ageProto="/content/drive/My Drive/Colab Notebooks/age_deploy.prototxt"
ageModel="/content/drive/My Drive/Colab Notebooks/age_net.caffemodel"
genderProto="/content/drive/My Drive/Colab Notebooks/gender_deploy.prototxt"
genderModel="/content/drive/My Drive/Colab Notebooks/gender_net.caffemodel"

MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)
# create the detector, using default weights
detector = MTCNN()

#data["image"] = ""
data["face_num"] = np.nan
data["age"] = np.nan
data["gender"] = np.nan
data["age_conf"] = ""
data["gender_conf"] = ""
data["face"] = ""
data["points"] = ""
padding=20

In [0]:
with progressbar.ProgressBar(max_value=len(data)) as bar:
    for i in range(18500, len(data)):
        bar.update(i)
        try:
            #skip if the user profile page no longer exists
            if(str(data.iloc[i,data.columns.get_loc("pic")])=="nan"):
                continue
            #skip if there's no profile pic
            if(data.pic[i]=="https://a0.muscache.com/defaults/user_pic-225x225.png?v=3"):
                continue
            image, face_ret, age_ret, gender_ret, agePreds, genderPreds, faceBoxes, points = analyze_pic(data.pic[i])
            #data.at[i, 'image'] = image
            data.loc[i, 'face_num'] = face_ret
            if face_ret == 1:
                data.loc[i, 'age'] = age_ret[0]
                data.loc[i, 'gender'] = gender_ret[0]
                data.at[i, 'age_conf'] = agePreds[0]
                data.at[i, 'gender_conf'] = genderPreds[0]
                data.at[i, 'face'] = faceBoxes[0]
                data.at[i, 'points'] = points
        #user profile page exists, but theres no profile picture
        except HTTPError:
            continue
        except Exception as e:
            # import pdb, traceback, sys
            # extype, value, tb = sys.exc_info()
            # traceback.print_exc()
            # pdb.post_mortem(tb)
            print("")
            print("index: "+str(i))
            print(str(e))
            print("=============================================================================")
        if(i%100==0 or i==len(data)-1):
            data.to_pickle("/content/drive/My Drive/Colab Notebooks/result_"+batch+".pkl")

 37% (18554 of 50000) |######            | Elapsed Time: 0:00:40 ETA:  17:11:50

In [0]:
data = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/result_"+batch+".pkl")

In [12]:
data['face_num'].index.get_loc(data['face_num'].last_valid_index())

18500

In [10]:
data[14310:14330]

,index,listing_id,id,pic,face_num,age,gender,age_conf,gender_conf,face,points
0,350000,17155934,490334885,https://a0.muscache.com/im/pictures/user/ab5bc...,NaN,NaN,NaN,,,,
1,350001,17155934,493894904,https://a0.muscache.com/im/pictures/user/eb1bf...,NaN,NaN,NaN,,,,
2,350002,17155934,511306329,https://a0.muscache.com/im/users/46224301/prof...,NaN,NaN,NaN,,,,
3,350003,17155934,517233707,https://a0.muscache.com/im/users/19205133/prof...,NaN,NaN,NaN,,,,
4,350004,17155934,527113564,https://a0.muscache.com/im/pictures/user/9e080...,NaN,NaN,NaN,,,,
...,...,...,...,...,...,...,...,...,...,...,...
49995,399995,20885139,196761454,https://a0.muscache.com/im/pictures/user/79e03...,NaN,NaN,NaN,,,,
49996,399996,20885139,199895446,https://a0.muscache.com/im/pictures/user/63b19...,NaN,NaN,NaN,,,,
49997,399997,20885139,202163280,https://a0.muscache.com/im/pictures/user/d6790...,NaN,NaN,NaN,,,,
49998,399998,20885139,202854498,https://a0.muscache.com/im/pictures/user/feb4c...,NaN,NaN,NaN,,,,
